In [1]:
env_USE_GOOGLE_COLAB = False
env_USE_NVIDIA = False
env_CONTENT_PATH = ""
env_TIF_PATH = ""
env_IMAGE_SHAPE_X = 0
env_IMAGE_SHAPE_Y = 0
env_MODEL_FILENAME = ""
env_FULL_MODEL_PATH = ""
import sys
from settings import load_BatGutsSettings, show_python_version
S = load_BatGutsSettings()
env_TIF_PATH = S.zTifPath
env_CONTENT_PATH = S.zContentPath
env_MODEL_FILENAME = S.zModelFilename
env_FULL_MODEL_PATH = env_CONTENT_PATH + "/" + env_MODEL_FILENAME
env_USE_GOOGLE_COLAB = S.useGoogleColab
env_USE_NVIDIA = S.useNvidia
env_IMAGE_SHAPE_X = S.imageShapeX
env_IMAGE_SHAPE_Y = S.imageShapeY
if len(env_CONTENT_PATH) == 0:
    sys.exit("Exiting the script because setings were not processed")

Loading settings from /Users/thomasyohe/isr/BatGutsImageClassification2/jupyternb/.env
my_CONTENT_PATH=/Users/thomasyohe/isr/bguts
my_TIF_PATH=/Users/thomasyohe/isr/bguts/tiffs


In [2]:
if env_USE_GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))

No GPU, using /device:CPU:0.


In [4]:
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff
from pyngrok import ngrok
import numpy as np
from PIL import Image as im
from PIL import  ImageOps
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
from tensorflow.python.keras.utils.data_utils import Sequence
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline
print(tf.__version__)

2.17.0


In [5]:
def display_tif_image(path):
  baseimage = tiff.imread(path)
  print ('original image shape',baseimage.shape)
  baseimage = np.squeeze(baseimage)

  plt.figure()

  plt.imshow(baseimage)

In [6]:
zBaseDir = env_TIF_PATH + "/LS042_Cormura_brevirostris-LY20_10-insect"
zDir = zBaseDir +"/LS042_Cormura_brevirostris-LY20_10-insect-guts-LY20-10A-AB-08"
full_path = zDir + "/LS042_Cormura_brevirostris-LY20_10-insect-guts-LY20-10A-AB-08-LY20-10A_ab_08_LS042_Cobr_2021-09-21 15-27-21 (B,Radius8,Smoothing4).tif"
display_tif_image(full_path)

original image shape (8184, 8248, 3)


In [7]:
zBase = env_TIF_PATH + "/DRXXX_Brachyphylla_pumila-LY20_13-nectar"
zDir = zBase + "/DRXXX_Brachyphylla_pumila-LY20_13-nectar-guts-LY20-13A-HE-014"
file_path = zDir + "/DRXXX_Brachyphylla_pumila-LY20_13-nectar-guts-LY20-13A-HE-014-LY20-13A-014.tif"
display_tif_image(file_path)

original image shape (5869, 5876, 3)


In [8]:
import seaborn as sns
import cv2
# plt.imshow(image)
import time
import os
import glob

In [9]:
#Standardizing the image
def standardize_image(img_list):
  mean_img = np.mean(img_list,axis=0)
  std_img = np.std(img_list,axis=0)
  img_list = (img_list - mean_img)/std_img
  return img_list


In [10]:
'''
This function is part of a machine learning pipeline for image classification.
It's designed to load images from a specific category and associate them with labels.
The use of one_hot_list suggests it's preparing data for a multi-class classification task.
The function can either start with empty lists or append to existing lists, allowing for batch processing. 
'''
def train_imgs(img_list=list(),category=None,one_hot_list=list(),label_list=list()):
    if len(img_list)==0 and len(one_hot_list)==0 and len(label_list)==0 :
        img_list=[]
        label_list=[]
    else:
        img_list =img_list
    for main_path in glob.glob( env_TIF_PATH + "/images/Bats_mixed_images_RGB_256/"+ str(category)+"/"):
        paths=glob.glob(os.path.join(main_path,"*.jpeg"))
        print('{} Images Count: {}'.format(category,len(paths)))
        for image_path in paths:
            image = cv2.imread(image_path)
            img_list.append(image)
            label_list.append(one_hot_list)
    return img_list,label_list







In [11]:
import os
import glob
import cv2

def train_imgs(img_list=None, category=None, label_list=None):
    img_list = img_list or []
    label_list = label_list or []
    
    file_path = os.path.join(env_CONTENT_PATH, "images", "Bats_mixed_images_RGB_256", str(category))
    
    if not os.path.exists(file_path):
        print(f"Warning: Directory not found for category {category}")
        return img_list, label_list
    
    image_paths = glob.glob(os.path.join(file_path, "*.jpeg"))
    print(f'{category} Images Count: {len(image_paths)}')
    
    for image_path in image_paths:
        image = cv2.imread(image_path)
        if image is not None:
            img_list.append(image)
            label_list.append(category)
        else:
            print(f"Warning: Could not read image {image_path}")
    
    return img_list, label_list

In [12]:
insects_list=list()
k=[]
insects_list,k=train_imgs(insects_list,'Insects',k)

plants_list = list()
plants_list,k=train_imgs(plants_list,'Plants',k)

len(plants_list[0:97])

Insects Images Count: 16
Plants Images Count: 30


30

In [13]:
#prepare training data
train_imgs_list = list()
labels_list =list()
# train_imgs_list,labels_list=train_imgs(train_imgs_list,'Blood',labels_list)
train_imgs_list,labels_list=train_imgs(train_imgs_list,'Insects',labels_list)

train_imgs_list,labels_list=train_imgs(train_imgs_list,'Plants',labels_list)


Insects Images Count: 16
Plants Images Count: 30


In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
labels_list= LabelEncoder().fit_transform(labels_list)

In [16]:
from collections import Counter
Counter(labels_list)

Counter({1: 30, 0: 16})

In [17]:
''' 
The purpose of this function is to shuffle two lists, train_imgs_list and labels_list, in unison. 
This is a common preprocessing step in machine learning to ensure that the data is randomized, 
which helps to prevent the model from learning any unintended patterns that might be 
present in the order of the data.

Why Shuffling is Important
Prevent Overfitting: Shuffling helps to prevent the model from overfitting to the order of the data.
Better Generalization: It ensures that the model generalizes better by learning from a 
more diverse set of examples in each epoch.
Balanced Batches: When training in batches, shuffling helps to ensure that each batch 
is representative of the entire dataset.

''' 
import random

temp = list(zip(train_imgs_list, labels_list)) # The zip function pairs elements from train_imgs_list 
                                                # and labels_list together into tuples.
random.shuffle(temp)
train_imgs_list,labels_list = zip(*temp) # The zip(*temp) expression unpacks the shuffled list of tuples back 
                                          # into two separate lists.
train_imgs_list,labels_list = list(train_imgs_list), list(labels_list)

In [18]:
li = len(train_imgs_list)
print("The length of the trained images list is " + str(li) )
ll = len(labels_list)
print("The length of the labels list is " + str( ll ))

The length of the trained images list is 46
The length of the labels list is 46


In [19]:
''' 
This code is performing two important preprocessing steps on a set of images, likely in preparation for t
raining a machine learning model, particularly a neural network. 
'''
X = np.array(train_imgs_list,np.float32)/255.0
''' 
The abpve line converts train_imgs_list (presumably a list of images) into a NumPy array.
It specifies the data type as np.float32 for efficient computation.
The division by 255.0 normalizes the pixel values to a range of [0, 1].
This normalization is common in image processing because pixel values typically 
range from 0 to 255 in standard image formats.
'''
X = standardize_image(X)
''' 
The above line calls a function named standardize_image on the normalized array X.
While we don't see the implementation of standardize_image, it's likely performing image standardization.
Standardization typically involves adjusting the values to have a mean of 0 and a standard 
deviation of 1 across the dataset.

'''

" \nThe above line calls a function named standardize_image on the normalized array X.\nWhile we don't see the implementation of standardize_image, it's likely performing image standardization.\nStandardization typically involves adjusting the values to have a mean of 0 and a standard \ndeviation of 1 across the dataset.\n\n"

In [20]:
''' 
This cell is performing the task of splitting a dataset into training and validation (or test) sets.
'''

''' 
Splitting the data into train and validation set

  X: This is the array of feature data (in this case, probably the images).

  labels_list: This is the array of labels corresponding to the feature data.

  test_size=0.1: This specifies that 10% of the data should be allocated to the test set, and the remaining 90% to the 
  training set.

  random_state=1: This ensures that the split is reproducible. Setting a random state means that every time you 
  run the code with the same data and the same random state, you will get the same split.

   stratify=labels_list: This ensures that the split preserves the proportion of each class in the labels. 
   This is particularly important for imbalanced datasets to ensure that both the training and test sets have a 
   similar distribution of classes.
'''
from sklearn.model_selection import train_test_split
X_train,x_test,Y_train,y_test = train_test_split(X,labels_list ,test_size=0.1,random_state=1,stratify=labels_list)



In [21]:
''' 
The make_pairs function is designed to create pairs of samples from a dataset, along with corresponding 
labels indicating whether the pairs are from the same class or different classes. This is often used in 
tasks such as training a Siamese network for similarity learning or one-shot learning.

Input Parameters
x: A list or array of data samples (e.g., images).
y: A list or array of labels corresponding to the data samples.

In summary, the make_pairs function is useful for creating training data for models that need to learn 
relationships or similarities between samples, such as Siamese networks. It ensures that the dataset contains both 
matching and non-matching pairs, which is crucial for training such models effectively.
'''
def make_pairs(x, y):
    num_classes = max(y) + 1
    #print("num_classes=" + str(num_classes) + " x=" + str(x) + "y=" + str(y)) 
    digit_indices = [np.where(y == i)[0] for i in range(num_classes)]
    #print("digit_indices="+str(digit_indices))
    pairs = []
    labels = []

    for idx1 in range(len(x)):
        #print(idx1)
        # add a matching example
        x1 = x[idx1]
        label1 = y[idx1]
        #print("label1="+str(label1))
        for i in range(5):
          #print("digit_indices[label1="+str(digit_indices[label1]))
          if digit_indices[label1]:  # Check if the list is not empty
            idx2 = random.choice(digit_indices[label1])
          else:
            # Handle the case where the list is empty
            #print(f"No indices available for label {label1}")
            # You might want to skip this iteration or choose a default value
            continue
          x2 = x[idx2]

          pairs += [[x1, x2]]
          labels += [1]

          # add a non-matching example
          label2 = random.randint(0, num_classes - 1)
          while label2 == label1:
              label2 = random.randint(0, num_classes - 1)

          idx2 = random.choice(digit_indices[label2])
          x2 = x[idx2]

          pairs += [[x1, x2]]
          labels += [0]
    a = np.array(pairs)
    b = np.array(labels).astype("float32")
    print("make_pairs returning pairs=" + str(pairs) + " np.array(pairs)=" + str(a) + " np.array(labels).astype(float32)="+str(b))
    return a, b


In [22]:
''' 
converts labels_list into a NumPy array. Let's break down its purpose and implications:

Purpose:
NumPy arrays offer several advantages over Python lists, especially in the context of machine 
learning and data processing:
a. Efficiency: NumPy arrays are more memory-efficient and faster to process than Python lists, especially for large datasets.
b. Vectorization: NumPy allows for vectorized operations, which can significantly speed up computations.
c. Compatibility: Many machine learning libraries and functions (like scikit-learn, TensorFlow, etc.) expect 
   inputs as NumPy arrays.
d. Multidimensional Support: NumPy arrays can easily represent multidimensional data, which is common in machine 
   learning tasks.
'''
labels_list = np.array(labels_list)

In [23]:
''' 
 make_pairs is probably designed to create pairs of images, which is a common technique used in certain types of 
 machine learning models, particularly in tasks like Siamese networks, Contrastive learning, One-shot learning. Face verification
'''
pairs_train, labels_train = make_pairs(X_train, Y_train)
print("Shape of pairs_train:", pairs_train.shape)
print("Shape of labels_train:", labels_train.shape)


make_pairs returning pairs=[] np.array(pairs)=[] np.array(labels).astype(float32)=[]
Shape of pairs_train: (0,)
Shape of labels_train: (0,)


/var/folders/fw/j27fkmm12hz_82xl8gd1v3tc0000gn/T/ipykernel_35684/4061704176.py:17: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  digit_indices = [np.where(y == i)[0] for i in range(num_classes)]
/var/folders/fw/j27fkmm12hz_82xl8gd1v3tc0000gn/T/ipykernel_35684/4061704176.py:30: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if digit_indices[label1]:  # Check if the list is not empty


In [24]:
pairval, labelval = make_pairs(x_test, y_test)
print("Shape of pairval:", pairval.shape)
print("Shape of labelval:", labelval.shape)

make_pairs returning pairs=[] np.array(pairs)=[] np.array(labels).astype(float32)=[]
Shape of pairval: (0,)
Shape of labelval: (0,)


/var/folders/fw/j27fkmm12hz_82xl8gd1v3tc0000gn/T/ipykernel_35684/4061704176.py:17: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  digit_indices = [np.where(y == i)[0] for i in range(num_classes)]
/var/folders/fw/j27fkmm12hz_82xl8gd1v3tc0000gn/T/ipykernel_35684/4061704176.py:30: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if digit_indices[label1]:  # Check if the list is not empty


In [25]:
X.shape

(46, 256, 256, 3)

In [26]:
train_image1 = np.array([pair[0] for pair in pairs_train])
train_image2 = np.array([pair[1] for pair in pairs_train])
#Below is Jaswanth's original code which get index errors

#train_image1 = pairs_train[:, 0]  # x_train_1.shape is (60000, 28, 28)
#train_image2 = pairs_train[:, 1]

In [27]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import plot_model

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
# import the necessary packages
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D

In [28]:
''' 
creates a convolutional neural network (CNN) model, typically used as one half of a Siamese network. 

Key Points:
This model is designed for feature extraction, typically used in Siamese networks for tasks like 
similarity learning or face recognition.

It uses a series of convolutional layers with decreasing filter sizes (256 -> 128 -> 64) to extract features.

Dropout layers are used for regularization to prevent overfitting.

The final dense layer creates an embedding of size embeddingDim.

The model doesn't include any classification layers, as it's meant to generate feature embeddings.

This architecture is suitable for learning a compact representation of input images, which can then be 
used to compare similarities between different inputs in a Siamese network setup.

'''
def build_siamese_model(inputShape, embeddingDim=256):
	# specify the inputs for the feature extractor network
	inputs = Input(inputShape) #Defines the input layer with the specified shape.
	
 	# define the first set of CONV => RELU => POOL => DROPOUT layers
  	#First Convolutional Block: 256 filters, 2x2 kernel, ReLU activation, Max pooling with 2x2 pool size30% dropout
	x = Conv2D(256, (2, 2), padding="same", activation="relu")(inputs)
	x = MaxPooling2D(pool_size=(2, 2))(x)
	x = Dropout(0.3)(x)
 
	# second set of CONV => RELU => POOL => DROPOUT layers
	#Second Convolutional Block: wotj similar pool and dropout attributes
	x = Conv2D(128, (2, 2), padding="same", activation="relu")(x)
	x = MaxPooling2D(pool_size=2)(x)
	x = Dropout(0.3)(x)
  
  # Third set of CONV => RELU => POOL => DROPOUT layers 
  # Third Convolutional Block: 64 filters and 40% dropout Reduces spatial dimensions to 1x1
	x = Conv2D(64, (2, 2), padding="same", activation="relu")(x)
	x = MaxPooling2D(pool_size=2)(x)
	x = Dropout(0.4)(x)
  
  # prepare the final outputs
	pooledOutput = GlobalAveragePooling2D()(x)
	outputs = Dense(embeddingDim)(pooledOutput)

	# build the model
	model = Model(inputs, outputs) #Creates a Keras Model with the defined inputs and outputs

	# return the model to the calling function
	return model

In [29]:
'''
calculates the Euclidean distance between pairs of vectors. It's commonly used in Siamese networks to measure 
the similarity (or dissimilarity) between two input samples. 
'''
def euclidean_distance(vectors):
	# unpack the vectors into separate lists
	(featsA, featsB) = vectors
	# compute the sum of squared distances between the vectors
	sumSquared = K.sum(K.square(featsA - featsB), axis=1,
		keepdims=True)
	# return the euclidean distance between the vectors
	return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [30]:
def plot_training(H, plotPath):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(H.history["loss"], label="train_loss")
	plt.plot(H.history["val_loss"], label="val_loss")
	plt.plot(H.history["accuracy"], label="train_acc")
	plt.plot(H.history["val_accuracy"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	plt.savefig(plotPath)

In [31]:
print("[INFO] building siamese network...")
imgA = Input(shape=(256,256,3))
imgB = Input(shape=(256,256,3))
featureExtractor = build_siamese_model((256,256,3))
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)

[INFO] building siamese network...


In [32]:
# finally, construct the siamese network
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)

In [33]:
# Defining callback Methods
n_epoch = 300

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1,
                           mode='auto', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5,
                             verbose=1, mode='auto')

In [34]:
# compile the model
# Check the shapes of the arrays
print("Shape of pairs_train:", pairs_train.shape)
print("Shape of labels_train:", labels_train.shape)
print("Shape of pairval:", pairval.shape)
print("Shape of labelval:", labelval.shape)

# Ensure pairs_train and pairval are 2-dimensional
if len(pairs_train.shape) == 1:
    pairs_train = pairs_train.reshape(-1, 2)
if len(pairval.shape) == 1:
    pairval = pairval.reshape(-1, 2)

# Check the shapes again after reshaping
print("Shape of pairs_train after reshaping:", pairs_train.shape)
print("Shape of pairval after reshaping:", pairval.shape)

# Compile the model
print("[INFO] compiling model...")
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
print("[INFO] training model...")
history = model.fit(
    [pairs_train[:, 0], pairs_train[:, 1]], labels_train[:],
    validation_data=([pairval[:, 0], pairval[:, 1]], labelval[:]),
    batch_size=10, epochs=n_epoch,
    callbacks=[reduce_lr, early_stop]
)

Shape of pairs_train: (0,)
Shape of labels_train: (0,)
Shape of pairval: (0,)
Shape of labelval: (0,)
[INFO] compiling model...
[INFO] training model...


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
# serialize the model to disk
print("[INFO] saving siamese model...")
model.save( env_FULL_MODEL_PATH )
# plot the training history
print("[INFO] plotting training history...")
plot_training(history, 'plot.png')

In [ ]:
from keras.models import load_model

print("[INFO] loading siamese model...")
model_path = env_FULL_MODEL_PATH
model = load_model(model_path)

In [ ]:
model.summary()

In [ ]:
file_base = env_CONTENT_PATH + "/images/Bats_mixed_images_RGB_256/Plants"
file_path = file_base + "/DRXXX_Brachyphylla_pumila-LY20_13-nectar-guts-LY20-13A-HE-006-Plane004__blue.jpeg"
imageA = cv2.imread(file_path)


In [ ]:
imageA.shape

In [ ]:
category = 'Plants'
file_prefix = env_CONTENT_PATH + "/images/"
for main_path in glob.glob( file_prefix + "Bats_mixed_images_RGB_256/"+ str(category)+"/"):
      paths=glob.glob(os.path.join(main_path,"*.jpeg"))
print('{} Images Count: {}'.format(category,len(paths)))

for i,image_path in enumerate(paths):
    imageA = cv2.imread(image_path)
    file_path = env_CONTENT_PATH + "/images/Bats_mixed_images_RGB_256/Plants/DRXXX_Brachyphylla_pumila-LY20_13-nectar-guts-LY20-13A-HE-006-Plane004__blue.jpeg"
    imageB = cv2.imread(file_path)
    # create a copy of both the images for visualization purpose
    origA = imageA.copy()
    origB = imageB.copy()
    imageA = np.expand_dims(imageA, axis=-1)
    imageB = np.expand_dims(imageB, axis=-1)
    # add a batch dimension to both images
    imageA = np.expand_dims(imageA, axis=0)
    imageB = np.expand_dims(imageB, axis=0)
    # scale the pixel values to the range of [0, 1]
    imageA = imageA / 255.0
    imageB = imageB / 255.0
    print(imageA.shape)
    # use our siamese model to make predictions on the image pair,
    # indicating whether or not the images belong to the same class
    preds = model.predict([imageA,imageB])
    proba = preds[0][0]
    fig = plt.figure("Pair #{}".format(i+1), figsize=(4, 2))
    plt.suptitle("Similarity: {:.2f}".format(proba))
    # show first image
    ax = fig.add_subplot(1, 2, 1)
    plt.imshow(origA, cmap=plt.cm.gray)
    plt.axis("off")
    # show the second image
    ax = fig.add_subplot(1, 2, 2)
    plt.imshow(origB, cmap=plt.cm.gray)
    plt.axis("off")
    # show the plot
    plt.show()

In [ ]:
# initialize the figure
fig = plt.figure("Pair #{}".format(1), figsize=(4, 2))
plt.suptitle("Similarity: {:.2f}".format(proba))
# show first image
ax = fig.add_subplot(1, 2, 1)
plt.imshow(origA, cmap=plt.cm.gray)
plt.axis("off")
# show the second image
ax = fig.add_subplot(1, 2, 2)
plt.imshow(origB, cmap=plt.cm.gray)
plt.axis("off")
# show the plot
plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE


In [ ]:
X.shape[0]

In [ ]:
X_re = X.reshape((X.shape[0],256*256*3))

In [ ]:
sm = SMOTE(random_state=42)
Xs_train, ys_train = sm.fit_resample(X_re, labels_list)

In [ ]:
Xs_train.shape

In [ ]:
Xs_train = Xs_train.reshape(-1,256,256,3)

In [ ]:
Xs_train.shape

In [ ]:
len(ys_train)

In [ ]:
# Counter(labels_list_smote)

In [ ]:
# len(labels_list_smote)

In [ ]:
# X_smote.shape

In [ ]:
# Xsmote=X_smote.reshape(X_smote.shape[0],256,256,3)

In [ ]:
# Xsmote.shape

In [ ]:
num_classes = 2
def one_hot_labels(img_list,classes):


  label_numeric = {k:v for v,k in enumerate(set(img_list))}
  target_values = [label_numeric[k] for k in img_list]
  target_values = np.array(target_values)
  target_values = to_categorical(target_values,num_classes=num_classes)
  return target_values

labels_list = one_hot_labels(labels_list,num_classes)

In [ ]:
# labels_list = np.array(labels_list)

In [ ]:
# len(labels_list_smote)

In [ ]:
# Splitting the data into train and validation set
from sklearn.model_selection import train_test_split
X_train,x_test,Y_train,y_test = train_test_split(Xs_train,ys_train ,test_size=0.1,random_state=1,stratify=ys_train)
x_train,x_valid,y_train,y_valid = train_test_split(X_train,Y_train,test_size=0.1,random_state=1,stratify=Y_train)

In [ ]:
''' 
This cell is performing a split of a dataset into training and validation sets. 

This approach is a form of train-validation split, which is common in machine learning to have a separate 
set for validating the model's performance during training. The specific choice of 2000 samples for validation 
is arbitrary and might be 
adjusted based on the total size of the dataset and the desired proportions for training and validation.
'''
LIMIT_VAL = 2000
X_train2 = []
y_train2 = []
X_val = Xs_train[:LIMIT_VAL]
y_val = ys_train[:LIMIT_VAL].reset_index(drop=True)
X_train2 = Xs_train[LIMIT_VAL:]
y_train2 = ys_train[LIMIT_VAL:].reset_index(drop=True)

In [ ]:
''' 
This cell is setting up an ImageDataGenerator from Keras for data augmentation. Data augmentation i
s a technique used to artificially expand a dataset by creating modified versions of existing images. 
This is particularly useful in deep learning for image classification tasks, 
as it helps to prevent overfitting and improves the model's ability to generalize.
'''
# Data Augumentation
train_datagen = ImageDataGenerator(
        rotation_range=10, # rotation
        width_shift_range=0.2, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        brightness_range=[0.2,1.2],# brightness
        fill_mode='nearest')


In [ ]:
''' 
compute the internal statistics of the dataset X that are needed for some of the data augmentation techniques. 
However, it's important to note that for most use cases of ImageDataGenerator, especially with the parameters 
you've set, this line is actually unnecessary and doesn't have any effect. 
'''
train_datagen.fit(X)

In [ ]:
#!pip install keras-unet

In [ ]:
# base_model = ResNet50(include_top=False,weights='imagenet',input_shape=(256,256,3))

In [ ]:
''' 
This cell is importing and creating an instance of a U-Net model, specifically a 
vanilla U-Net, using the keras_unet library. 

U-Net Model Overview
The U-Net architecture is widely used for image segmentation tasks. It consists of two main parts:
Contracting Path (Encoder):
This part of the network captures context and features from the input image.
It typically consists of a series of convolutional layers followed by max-pooling layers, 
which progressively reduce the spatial dimensions and increase the depth of the feature maps.
Expanding Path (Decoder):
This part of the network reconstructs the spatial dimensions of the image while combining the high-level 
features captured by the encoder.
It typically consists of upsampling layers (like transposed convolutions) and concatenation with 
corresponding feature maps from the encoder (skip connections).


'''
from keras_unet.models import vanilla_unet

base_model  = vanilla_unet(input_shape=(256, 256, 3))

In [ ]:
''' 
This code is creating a new model by adding additional layers on top of a pre-existing base model 
(likely the U-Net model you created earlier). 

What this code accomplishes:
Transfer Learning: It's using the base model (likely the U-Net) as a feature extractor.
Classification Head: It's adding a classification head on top of the base model, turning what was 
 likely a segmentation model into a classification model.
Binary Classification: The final layer with 2 units and softmax activation suggests this is 
  now set up for binary classification.
Regularization: Dropout layers are added to prevent overfitting.
Model Composition: It demonstrates how to combine a pre-trained model with additional custom layers.
'''
from tensorflow.keras.layers import BatchNormalization
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256,activation='relu'))
add_model.add(Dropout(0.3))
add_model.add(Dense(256,activation='relu'))
add_model.add(Dropout(0.5))
add_model.add(Dense(2,activation='softmax'))
model = Model(inputs=base_model.input,outputs = add_model(base_model.output))
model.summary()

In [ ]:
# Compiling Model
optimizer = Adam(learning_rate=1e-5)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
# Defining callback Methods
n_epoch = 60

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1,
                           mode='auto', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5,
                             verbose=1, mode='auto')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
for idx, batch_size in enumerate([8, 16,32,64,128]):
    history = model.fit(train_datagen.flow(x_train,y_train,batch_size=10),
                    epochs=n_epoch,
                    callbacks=[reduce_lr,early_stop],
                    validation_data=(x_valid,y_valid),steps_per_epoch=batch_size
                   )
    # Plotting the results on Graph
    fig, ax = plt.subplots(2,1)
    ax[0].plot(history.history['loss'], color='b', label="Training loss")
    ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
    legend = ax[0].legend(loc='best', shadow=True)

    ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
    ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
    legend = ax[1].legend(loc='best', shadow=True)

    predictions_model = model.predict(x_test)
    predictions_model_max = np.argmax(predictions_model,axis=1)
    predictions_onehot = to_categorical(predictions_model_max)
    accuracy = accuracy_score(y_test,predictions_onehot)

    print(classification_report(y_test,predictions_onehot,target_names=['Insects','Plants']))

    # Start MLflow
    RUN_NAME = f"run_{idx}"
    with mlflow.start_run(experiment_id=EXPERIMENT_ID, run_name=RUN_NAME) as run:
        # Retrieve run id
        RUN_ID = run.info.run_id

        # Track parameters
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("epochs", 60)
        mlflow.log_param("image_size", 256)


        # Track model
        mlflow.sklearn.log_model(model, "Bat_guts_IVP_classifier")

        #Track confusion matrix results
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric('Weighted Precision',(precision_score(y_test,predictions_onehot, average='weighted')))
        mlflow.log_metric('Weighted Recall',(recall_score(y_test,predictions_onehot, average='weighted')))
        mlflow.log_metric('Weighted F1-score',(f1_score(y_test,predictions_onehot, average='weighted')))


In [ ]:
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2AVGMje37gfu1yzYm2UJWam40Mm_dmYa8aw82QJvUCwSCETo"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(port="5000", proto="http", options={"bind_tls": True})
print("MLflow Tracking UI:", ngrok_tunnel)

In [ ]:
!mlflow ui --backend-store-uri /content/drive/MyDrive/mlruns

In [ ]:
# Plotting the results on Graph
#fig, ax = plt.subplots(2,1)
#ax[0].plot(history.history['loss'], color='b', label="Training loss")
#ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
#legend = ax[0].legend(loc='best', shadow=True)

#ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
#ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
#legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
# Predictions on testing data created from training data
predictions_model = model.predict(x_test)

In [ ]:
predictions_model_max = np.argmax(predictions_model,axis=1)
predictions_onehot = to_categorical(predictions_model_max)

In [ ]:
y_test

In [ ]:
predictions_onehot

In [ ]:
# Classification Report
print(classification_report(y_test,predictions_onehot,target_names=['Insects','Plants']))

In [ ]:
from keras.applications.xception import preprocess_input,decode_predictions
import keras
print("keras version="+str(keras.__version__))

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
def classify(img_path,model):
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)

    img_batch = np.expand_dims(img_array, axis=0)

    img_preprocessed = preprocess_input(img_batch)

    prediction = model.predict(img_preprocessed)
    print(prediction)
    prediction_max = [int(i > .5) for i in prediction[0]]

    if 1 in prediction_max and sum(prediction_max)==1:
      # if prediction_max.index(1) ==0:
      #   return 'Bat eats Blood'
      if prediction_max.index(1) ==0:
        return 'This Bat eat Insects'
      if prediction_max.index(1) ==1:
        return 'This Bat eat Plants'

    else:
      return prediction_max

In [ ]:

logged_model = 'runs:/b78dc243ebf244afa46734d35388620a/Bat_guts_IVP_classifier'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
import pickle
import numpy as np

In [ ]:
file_path = env_CONTENT_PATH + "/mlruns/0/271bf38e306e4da48d33b43c21fd79de/artifacts/Bat_guts_IVP_classifier/model.pkl"
loaded_model = pickle.load(open(file_path, 'rb'))

In [ ]:
file_path = env_CONTENT_PATH + "/images/Bats_mixed_images_RGB_256/Plants/DRXXX_Brachyphylla_pumila-LY20_13-nectar-guts-LY20-13A-HE-006-Plane004__blue.jpeg"
classify(file_path,loaded_model)

In [ ]:
file_path = env_CONTENT_PATH + "/images/Bats_mixed_images_RGB_256/Plants/Y20-14C-003__blue.jpeg"
classify(file_path,loaded_model)

In [ ]:
file_prefix = env_CONTENT_PATH +"/images/"
paths = glob.glob(os.path.join( file_prefix + "Bats_mixed_images_RGB_256/"+ str('Plants')+"/","*.jpeg"))


In [ ]:
import pickle

In [ ]:
file_path = env_CONTENT_PATH + "/model.pkl"
loaded_model = pickle.load(open( file_path, 'rb'))

In [ ]:
len(paths)

In [ ]:
k=0
for path in paths:

  if 'Bat eats Insects' == classify(path,loaded_model):
    print('here')
    k=k+1
  else:
    print(path)



In [ ]:
k,len(paths)

In [ ]:
j=0
for path in paths:

  if 'Bat eats Plants' == classify(path,loaded_model):
    print('here')
    j=j+1
  else:
    print(path)

In [ ]:
j,len(paths)

In [ ]:
#check versions
import flask
print( "flask version="+str(flask.__version__))
print("np version="+str(np.__version__))

show_python_version()
import mlflow
print("mlflow version="+str(mlflow.__version__))